# Which multidispatch is the fastest

## Single Dispatch

In [22]:
from functools import singledispatch
from simplegeneric import generic
import numpy as np

In [23]:
def raw(array):
    return np.sum(np.sin(array))

@singledispatch
def singled(*args, **kwargs):
    raise NotImplemented(f"{args}, {kwargs}")

@singled.register(int)
def _(a): print('int')

@singled.register(str)
def _(a): print('str')

@singled.register(np.ndarray)
def _(a): return np.sum(np.sin(a))

@generic
def simpleg(*args, **kwargs):
    raise NotImplemented(f"{args}, {kwargs}")

@simpleg.when_type(int)
def _(a): print('int')

@simpleg.when_type(str)
def _(a): print('str')

@simpleg.when_type(np.ndarray)
def _(a): return np.sum(np.sin(a))

In [24]:
np.random.seed(10)
a = np.random.rand(100000)

In [25]:
singled(a) == simpleg(a) == raw(a)

True

In [26]:
%timeit raw(a)

151 µs ± 5.46 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [27]:
%timeit singled(a)

154 µs ± 4.99 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [28]:
%timeit simpleg(a)

156 µs ± 4.07 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Multi-dispatch

In [29]:
from multipledispatch import dispatch
from functools import partial
import numpy as np
my_namespace = dict()
dispatch = partial(dispatch, namespace=my_namespace)


In [30]:
@dispatch(int)  # Uses my_namespace rather than the global namespace
def multi(x):
    print('int')
@dispatch(str)  # Uses my_namespace rather than the global namespace
def multi(x):
    print('str')
@dispatch(np.ndarray)  # Uses my_namespace rather than the global namespace
def multi(x):
    return np.sum(np.sin(x))

In [31]:
%timeit multi(a)

157 µs ± 5.75 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [32]:
@dispatch(int, int)
def multi(x, y):
    print('int int')
@dispatch(int, str)
def multi(x, y):
    print('int str')
@dispatch(str, int)
def multi(x, y):
    print('str int')
@dispatch(np.ndarray, np.ndarray)
def multi(x, y):
    return np.sum(np.sin(x).dot(np.cos(y)))

raw_multi = lambda x,y: np.sum(np.sin(x).dot(np.cos(y)))

In [33]:
multi(a,a) == raw_multi(a,a)

True

In [34]:
%timeit multi(a,a)

271 µs ± 7.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [35]:
%timeit raw_multi(a,a)

273 µs ± 6.07 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [36]:
my_namespace

{'multi': <dispatched multi>}